In [ ]:
%%capture
import numpy as np
import pandas as pd
import re
import random
!pip install pymorphy2
import pymorphy2
import dill

import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.text as text
import matplotlib.cm as cm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn.svm import SVC

from google.colab import drive
drive.mount('/content/gdrive')
# Исправьте путь до папки на вашем гугл диске:
gc_path = r"/content/gdrive/MyDrive/hackathon_classifier_of_university_hotline_messages/"

# Входные тренировочные данные. 

In [1]:
df = pd.read_excel(gc_path+'data_train_1000.xlsx')

NameError: ignored

In [ ]:
num_classes = 15
class_names = ['Сроки','Документы','Места','Проходной и допустимый балл','Достижения','Льготы','Общежития','Оплата','Другие вопросы','Аспирантура','Перевод','Ошибки','Забрать заявление','Магистратура','Очно-заочное и зачное']

In [ ]:
stop_words_greetings = ['спасибо','подскажите','расскажите','скажите','объясните','пожалуйста', 'здравствуйте','здравствуйте!','добрый день','до свидания', 'доброго времени суток', 'доброго дня', 'hello', 'С уважением']
cls_dic = {1:['Верный класс'],
           0:['Неверный класс']}

## Преобразование меток классов, например, из 2 5 в 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 

In [ ]:
def OHE_labels(df):
  temp = []
  for cl in df['CLASS']:
    new_type = np.array([0]*num_classes)
    classes = cl.split(' ')
    for i in classes:
      new_type[int(i)] = 1
    temp.append(new_type)
  y = np.array(temp)
  y = y.transpose()

  for nom, class_name in enumerate(['Сроки','Документы','Места','Проходной и допустимый балл','Достижения','Льготы','Общежития','Оплата','Другие вопросы','Аспирантура','Перевод','Ошибки','Забрать заявление','Магистратура','Очно-заочное и зачное']):
    df[class_name] = y[nom]
  return df

In [ ]:
df = OHE_labels(df)

# Подзадача 1. Выбор/Разработка методов предварительной обработки текста
## Подготовка данных

In [ ]:
def re_gr(txt):
    '''
    Удаление приветсвий и стоп-слов из строки.
    '''
    for x in stop_words_greetings:
        txt = txt.replace(x,'')
    return txt

def del_punct(txt):
    '''
    Удаляет знаки препинания и подчеркивания.
    '''
    res = re.sub(r'[^\w\s]',' ',txt)
    res = res.replace('_','')
    res = res.replace('\n','')
    return res

def morphan(word, morph):
    '''
    Приведение слов в нормальную форму, удаление числительных и ФИО.
    '''
    word = del_punct(word).strip()
    p = morph.parse(word)[0]
   
    word_new = word    
    if (not 'Surn' in p.tag) and (not 'Name' in p.tag) and (not 'Patr' in p.tag) and ('NOUN' in p.tag): 
        #существительное не ФИО
        word_new = p.normal_form
     
    elif 'Surn' in p.tag:
        word_new = 'ФАМИЛИЯ'
    elif 'Name' in p.tag:
        word_new = 'ИМЯ'
    elif 'Patr' in p.tag:
        word_new = 'ОТЧЕСТВО'
       
    elif ('INFN' in p.tag) or ('VERB' in p.tag): #глагол
        word_new = p.normal_form
                
    elif ('ADJF' in p.tag) or ('ADJS' in p.tag) or ('COMP' in p.tag): #прилагательное
        word_new = p.normal_form
            
    elif ('PRTF' in p.tag) or ('PRTS' in p.tag) or ('GRND' in p.tag): #причастие, похоже на глагол
        word_new = p.normal_form
                
    elif ('ADVB' in p.tag) or ('NPRO' in p.tag) or ('PRED' in p.tag) or ('PREP' in p.tag) or ('CONJ' in p.tag) or ('PRCL' in p.tag) or ('INTJ' in p.tag):
        # предлоги, местоимения и пр.
        word_new = p.normal_form         
  
    elif ('NUMR' in p.tag) or ('NUMB' in p.tag) or ('intg' in p.tag): # числительные NUMB,intg
        word_new = ''
        
    else:
        word_new = word

    return word_new

def remove2(txt):
    '''
    :txt - текст
    Возвращает слова, в которых не менее 2 символов.
    '''
    return ' '.join([x if len(x)>2 else '' for x in txt.split()])

def normtext(txt, morph):
    '''
    Возвращает текст из слов в нормальной форме
    '''
    return str(' '.join([morphan(x, morph) for x in txt.split()]))


In [ ]:
def prepare_dataframe(df):
  df['text'] = df['CONTENT'].map(lambda x: x.lower()).map(re_gr)
  df['text'] = df.text.map(del_punct)
  morph = pymorphy2.MorphAnalyzer()
  df['text'] = df.text.map(lambda x: normtext(x, morph))
  df['text'] = df.text.map(remove2)
  return df

In [ ]:
df = prepare_dataframe(df)

# Подзадача 2. Выбор/Разработка способа векторизации текста
## Tfidf Vectorizer (получение векторного представления)


In [ ]:
tfv = TfidfVectorizer()  # Функция получения векторного представления
tfv.fit(df.text)

TfidfVectorizer()

In [ ]:
# Функция для нахождения оптимальных значений параметров классификатора:

def get_best_classifier(X_train, y_train, classifier, param_grid):
  X_train = tfv.transform(X_train)  
  clf = GridSearchCV(classifier, param_grid, scoring='f1', n_jobs=-1) #, cv=3)
  clf.fit(X_train, y_train)
  return clf.best_estimator_

In [ ]:
def predictor(text, clf, tfv):
    '''
    Предсказывает класс сообщения
    :param text: классифицируемый текс
    :param clf: обученный классификатор
    :param tfv: обученный векторизатор
    '''
    X_test = tfv.transform([text])
    
    pred = clf.predict(X_test)
    #print(pred)
    return int(pred[0])

In [ ]:
# Сериализация / десериализция модели
def serialize(obj, filename):
    '''
    Сериализация объекта в файл
    '''
    output = open(filename, 'wb')
    dill.dump(obj, output)
    output.close()
    return
 
def deserialize(filename):
    '''
    Десериализация объекта из файла
    '''
    input = open(filename, 'rb')
    obj = dill.load(input)
    input.close()
    return obj

# Классификация

In [ ]:
def get_binary_datasets(df, num):
  '''
  :param df: Входной DataFrame
  :param num: Искомый (верный) класс
  :return Функция возвращает новый Binary DataFrame, в котором 1 означает, что сообщение относиться к соотвествующему классу, 0 - иначе
  '''
  temp_Content = []
  temp_Class = []

  for i, cl in enumerate(df['CLASS']):
    classes_str = cl.split(' ')
    classes_int = [int(x) for x in classes_str]
    temp_Content.append(df.iloc[i][18])  # 18 - номер столбца с обработанным сообщением
    if num in classes_int:
      temp_Class.append(1)
    else:
      temp_Class.append(0)

  content = np.array(temp_Content)
  classes = np.array(temp_Class)

  data = pd.concat([pd.DataFrame(content), pd.DataFrame(classes)], axis=1)
  data.columns = ['CONTENT','CLASS']

  return data

## Делим входной набор данных на тренировочные и валидацонные данные

In [ ]:
X_train, X_valid, y_train_del, y_valid_del = train_test_split(df, df['CLASS'], random_state=42, test_size=0.2) 

In [ ]:
train_binary_dataframes = []
valid_binary_dataframes = []
for i in range(num_classes):
  train_binary_dataframes.append(get_binary_datasets(X_train,i))
  valid_binary_dataframes.append(get_binary_datasets(X_valid,i))

# Разработка классификатора сообщений на основе одного метода ML/DL, настройка параметров классификатора

## Обучение моделей

### Классификаторы: Boosting, LightGBM’s 


In [ ]:
# # explore lightgbm number of trees effect on performance
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from lightgbm import LGBMClassifier
# from matplotlib import pyplot

# # get the dataset
# def get_dataset():
#     X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
#     return X, y

# # get a list of models to evaluate
# def get_models():
#     models = dict()
#     trees = [10, 50, 100, 500, 1000, 5000]
#     for n in trees:
#         models[str(n)] = LGBMClassifier(n_estimators=n)
#     return models

# # evaluate a give model using cross-validation
# def evaluate_model(model):
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#     scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#     return scores

# # define dataset
# X, y = get_dataset()

# # get the models to evaluate
# models = get_models()

# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

# # plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()


### Классификаторы: Boosting, CatBoostClassifier (база)


In [ ]:
!pip install catboost

import numpy as np
from catboost import CatBoostClassifier, Pool

# initialize data
train_data = np.random.randint(0, 100, size=(100, 10))
train_labels = np.random.randint(0, 2, size=(100))
test_data = catboost_pool = Pool(train_data, train_labels)

model = CatBoostClassifier(iterations=2,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

# train the model
model.fit(train_data, train_labels)

# make the prediction using the resulting model
preds_class = model.predict(test_data)
preds_proba = model.predict_proba(test_data)
print("class = ", preds_class)
print("proba = ", preds_proba)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
0:	learn: 0.6511534	total: 407us	remaining: 407us
1:	learn: 0.6230029	total: 1.96ms	remaining: 0us
class =  [1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0
 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 1 1 0]
proba =  [[0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.66791067 0.33208933]
 [0.48983394 0.51016606]
 [0.42891552 0.57108448]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.66791067 0.33208933]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.48983394 0.51016606]
 [0.4

### Habr


In [ ]:
# !pip install catboost
# from catboost import CatBoostClassifier

# X_train = None
# y_train = None
# cat_features = None

# model = CatBoostClassifier(
#     iterations=150,
#     random_seed=43,
#     loss_function='MultiClass'
# )

# model.fit(
#     X_train, y_train,
#     cat_features=cat_features,
#     eval_set=(X_test, y_test),
#     verbose=False,
#     plot=True
# )

## Тестируем параметры

In [ ]:
for nom in range(num_classes):
  
  # Задаем классифкатор
  classifier = SVC(max_iter=-1, kernel='rbf', gamma=0.01, C=1000, cache_size=200, class_weight=None, coef0=0.0, degree=3, probability=False, shrinking=True, tol=0.001)  # !!! Эксперименты
  
  # Задаем словарь парамтеров с их значениями для перебора
  param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')}  # !!! Эксперименты
  
  # Находим оптимальный классифкатор (класификтор с оптимальным значением параметров)
  clf = get_best_classifier(train_binary_dataframes[nom].CONTENT, train_binary_dataframes[nom].CLASS, classifier, param_grid)
  clf_tfv = {'clf':clf,'tfv':tfv}
  serialize(clf_tfv, gc_path + f'models/model_{nom}_class.pkl')

  # Предсказываем
  pred = []
  for txt in valid_binary_dataframes[nom].CONTENT: # .values):
    pred.append(predictor(txt, clf, tfv))

  # Выводим метрики классификации каждого бинарного классификатора
  mtrs = metrics.classification_report([cls_dic[x][0] for x in valid_binary_dataframes[nom].CLASS], [cls_dic[x][0] for x in pred])
  print(f"Metrics for {nom} class")
  print(mtrs, end='\n')
  

Metrics for 0 class
                precision    recall  f1-score   support

  Верный класс       0.58      0.32      0.41        22
Неверный класс       0.92      0.97      0.94       172

      accuracy                           0.90       194
     macro avg       0.75      0.64      0.68       194
  weighted avg       0.88      0.90      0.88       194

Metrics for 1 class
                precision    recall  f1-score   support

  Верный класс       0.46      0.24      0.32        25
Неверный класс       0.90      0.96      0.93       169

      accuracy                           0.87       194
     macro avg       0.68      0.60      0.62       194
  weighted avg       0.84      0.87      0.85       194

Metrics for 2 class
                precision    recall  f1-score   support

  Верный класс       0.80      0.44      0.57         9
Неверный класс       0.97      0.99      0.98       185

      accuracy                           0.97       194
     macro avg       0.89      0.72 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for 5 class
                precision    recall  f1-score   support

  Верный класс       0.00      0.00      0.00         3
Неверный класс       0.98      1.00      0.99       191

      accuracy                           0.98       194
     macro avg       0.49      0.50      0.50       194
  weighted avg       0.97      0.98      0.98       194

Metrics for 6 class
                precision    recall  f1-score   support

  Верный класс       1.00      0.60      0.75         5
Неверный класс       0.99      1.00      0.99       189

      accuracy                           0.99       194
     macro avg       0.99      0.80      0.87       194
  weighted avg       0.99      0.99      0.99       194

Metrics for 7 class
                precision    recall  f1-score   support

  Верный класс       0.00      0.00      0.00         7
Неверный класс       0.96      0.99      0.98       187

      accuracy                           0.96       194
     macro avg       0.48      0.50 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


Metrics for 9 class
                precision    recall  f1-score   support

  Верный класс       1.00      0.50      0.67         2
Неверный класс       0.99      1.00      1.00       192

      accuracy                           0.99       194
     macro avg       1.00      0.75      0.83       194
  weighted avg       0.99      0.99      0.99       194

Metrics for 10 class
                precision    recall  f1-score   support

  Верный класс       0.75      0.55      0.63        11
Неверный класс       0.97      0.99      0.98       183

      accuracy                           0.96       194
     macro avg       0.86      0.77      0.81       194
  weighted avg       0.96      0.96      0.96       194

Metrics for 11 class
                precision    recall  f1-score   support

  Верный класс       0.64      0.37      0.47        19
Неверный класс       0.93      0.98      0.96       175

      accuracy                           0.92       194
     macro avg       0.79      0.6

# Многометочная классификация

In [ ]:
# Т.к. вопрорсов по аспирантуре мало, и в них в основном встречается подстрока 'аспирант', то можно записи проклассифицировать по наличию этой подстроки
def find_aspirant(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'аспирант'
    '''
    return 'аспирант' in [x[:8] for x in txt.split()]

In [ ]:
def find_gold(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'золот'
    '''
    return 'золот' in [x[:5] for x in txt.split()]

def find_serebro(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'серебр'
    '''
    return 'серебр' in [x[:6] for x in txt.split()]

def find_sirot(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'сирот'
    '''
    return 'сирот' in [x[:5] for x in txt.split()]

def find_dostish(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'достижен'
    '''
    return 'достижен' in [x[:8] for x in txt.split()]

def find_olimp(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'олимпиад'
    '''
    return 'олимпиад' in [x[:8] for x in txt.split()]

def find_zaochn(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'очно-заочн'
    '''
    return 'очно-заочн' in [x[:10] for x in txt.split()]

def find_perevod(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'перевод'
    '''
    return 'перевод' in [x[:7] for x in txt.split()]

def find_ancity(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'иногород'
    '''
    return 'иногород' in [x[:8] for x in txt.split()]

def find_obshe(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'общежит'
    '''
    return 'общежит' in [x[:7] for x in txt.split()]


def find_pasport(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'паспорт'
    '''
    return 'паспорт' in [x[:7] for x in txt.split()]


def find_copy(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'копи'
    '''
    return 'копи' in [x[:4] for x in txt.split()]

## Возможно так и с другими классами поступить можно (как с классом "аспирантура")?

# Валидация модели

In [ ]:
# Функция, которая предсказывает все метки сообщений
def pred_labels(messages=None):
  preds = []
  for txt in messages:
    pred = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    skip = []
    if find_aspirant(txt):
      skip.append(9)
      pred[9] = 1

    if find_gold(txt):
      skip.append(4)
      pred[4] = 1
    if find_serebro(txt):
      skip.append(4)
      pred[4] = 1
    
    if find_sirot(txt):
      skip.append(5)
      pred[5] = 1
    if find_dostish(txt):
      skip.append(4)
      pred[4] = 1
    if find_olimp(txt):
      skip.append(4)
      pred[4] = 1

    if find_ancity(txt):
      skip.append(10)
      pred[10] = 1

    if find_zaochn(txt):
      skip.append(14)
      pred[14] = 1
    if find_perevod(txt):
      skip.append(10)
      pred[10] = 1

    if find_obshe(txt):
      skip.append(6)
      pred[6] = 1
    if find_copy(txt):
      skip.append(1)
      pred[1] = 1
    if find_pasport(txt):
      skip.append(1)
      pred[1] = 1

    for i in range(num_classes):
      if i in skip:
        continue
      clf_tfv = deserialize(gc_path + f'models/model_{i}_class.pkl')
      if predictor(txt, clf_tfv['clf'], clf_tfv['tfv']) == 1:
        pred[i] = 1
      else:
        pred[i] = 0
    preds.append(pred)
  return preds

In [ ]:
# Предсказываем все метки сообщений
preds = pred_labels(messages=X_valid.text)

In [ ]:
true_labels = X_valid[class_names].values

## Матрицы ошибок. Раскомментировать при необходимости:

## Таблица

In [ ]:
# from sklearn.metrics import multilabel_confusion_matrix
# vis_arr = multilabel_confusion_matrix(true_labels, preds)

# labels = ["".join("" + str(i)) for i in range(1, 16)]

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# vis_arr[7]

In [ ]:
# from sklearn.metrics import multilabel_confusion_matrix
# vis_arr = multilabel_confusion_matrix(true_labels, preds)

# labels = ["".join("" + str(i)) for i in range(1, 16)]

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns


# def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=18):

#     df_cm = pd.DataFrame(
#         confusion_matrix, index=class_names, columns=class_names,
#     )

#     try:
#         heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
#         sns.set(font_scale = 1.6)
#     except ValueError:
#         raise ValueError("Confusion matrix values must be integers.")
#     heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
#     heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
#     axes.set_ylabel('True class', fontsize=fontsize)
#     axes.set_xlabel('Predicted class', fontsize=fontsize)
#     axes.set_title("class_" + class_label, fontsize=fontsize)
  

# fig, ax = plt.subplots(15,1,figsize=(4, 15))
# for axes, cfs_matrix, label in zip(ax.flatten(), vis_arr[0:15], labels[0:15]):
#     print_confusion_matrix(cfs_matrix, axes, label, ["False", "True"])
# fig.subplots_adjust(left=None, bottom=1, right=None, top=3, wspace=None, hspace=2)
# fig.tight_layout()
# plt.show()

## Дерево

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np; np.random.seed(42)
# from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.datasets import make_circles # Make a large circle containing a smaller circle in 2d (toy dataset)
# from sklearn.model_selection import train_test_split
# plt.style.use('ggplot')
# plt.rcParams['figure.figsize'] = 10, 6

# # X - координаты центров кругов, y={0;1} - метки классов -> бинарная классификция
# X, y = make_circles(n_samples=500, factor=0.1, noise=0.35, random_state=42)
# X_train_circles, X_test_circles, y_train_circles, y_test_circles = train_test_split(X, y, test_size=0.2)

# dtree = DecisionTreeClassifier(random_state=42)
# dtree.fit(X_train_circles, y_train_circles)

# # Получаем новые центры кругов
# x_range = np.linspace(X.min(), X.max(), 100)
# xx1, xx2 = np.meshgrid(x_range, x_range) # xx1.shape=(100,100), xx2.shape=(100,100)

# # Классифицируем круги
# y_hat = dtree.predict(np.c_[xx1.ravel(), xx2.ravel()]) # y_hat.shape = 10000
# y_hat = y_hat.reshape(xx1.shape) # y_hat.shape = (100, 100)

# # Отрисовка кругов
# plt.contourf(xx1, xx2, y_hat, alpha=0.2) # Отрисовать область, разграничивающую два класса
# plt.scatter(X[:,0], X[:,1], c=y, cmap='autumn') # Отрисовать круги
# plt.title("Дерево решений")
# plt.show()

# b_dtree = BaggingClassifier(DecisionTreeClassifier(),n_estimators=300, random_state=42)
# b_dtree.fit(X_train_circles, y_train_circles)

# x_range = np.linspace(X.min(), X.max(), 100)
# xx1, xx2 = np.meshgrid(x_range, x_range)
# y_hat = b_dtree.predict(np.c_[xx1.ravel(), xx2.ravel()])
# y_hat = y_hat.reshape(xx1.shape)
# plt.contourf(xx1, xx2, y_hat, alpha=0.2)
# plt.scatter(X[:,0], X[:,1], c=y, cmap='autumn')
# plt.title("Бэггинг(дерево решений)")
# plt.show()

# rf = RandomForestClassifier(n_estimators=300, random_state=42)
# rf.fit(X_train_circles, y_train_circles)

# x_range = np.linspace(X.min(), X.max(), 100)
# xx1, xx2 = np.meshgrid(x_range, x_range)
# y_hat = rf.predict(np.c_[xx1.ravel(), xx2.ravel()])
# y_hat = y_hat.reshape(xx1.shape)
# plt.contourf(xx1, xx2, y_hat, alpha=0.2)
# plt.scatter(X[:,0], X[:,1], c=y, cmap='autumn')
# plt.title("Случайный лес")
# plt.show()

## В3

In [ ]:
# !pip install shap
# import shap
# shap.initjs()
# shap.force_plot(explainer.expected_value,shap_values[91, :], X.iloc[91, :])

import shap

explainer = shap.TreeExplainer(model)
shap_values=explainer.shap_values(Pool(X, y, cat_features=cat_features))

shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[3,:], X.iloc[3,:])


## Результаты

In [ ]:
def get_f1_score(true_labels, preds):
  f1 = []
  y_true_all = np.array(true_labels)
  y_pred_all = np.array(preds)
  for i in range(num_classes):
    y_true = y_true_all[:, i]
    y_pred = y_pred_all[:, i]
    f1.append(metrics.f1_score(y_true, y_pred, average='weighted'))
  print('FinAL f1 = ', sum(f1)/len(f1))

In [ ]:
get_f1_score(true_labels, preds)

FinAL f1 =  0.9366580664448778


1.   FinAL f1 =  0.9300064833617011 (первый запуск)
2.   FinAL f1 =  0.9310326099101509 (после прописания аспирант, 21:59)
3.   FinAL f1 =  0.9310326099101509 (после + cat bost)
4.   FinAL f1 =  0.9316154652436306 (изменили класссификатор)
5.   FinAL f1 =  0.9329342620456921 (изменили класссификатор)
6.   FinAL f1 =  0.5807435799345196 (изменили cat bost)
7.   FinAL f1 =  0.9350442682401167 (изменили класссификатор)
8.   FinAL f1 =  0.9422504217318218 (, kernel='rbf', gamma=0.01, C=1000)
9.   FinAL f1 =  0.907323843174683 param_grid = {"C": [0.1, 1, 10, 100, 1000], "gamma": [1, 0.1, 0.01, 0.001, 0.0001], "kernel": ["rbf"]}
10.  FinAL f1 =  0.9400340912631852   param_grid = {"C": [0.1, 1, 10, 100, 1000], "gamma": [1, 0.1, 0.01, 0.001, 0.0001], "kernel": ["rbf"]}  # !!! Эксперименты
11. FinAL f1 =  0.9395064069678872 - param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]




# Тестирование модели
! Файл для тестирования data_test.xlsx будет выложен на гугл диск задачи 7 августа в 8:30.

In [ ]:
df_test = pd.read_excel(gc_path+'data_test.xlsx')
df_test = prepare_dataframe(df_test)
df_test = OHE_labels(df_test)

In [ ]:
test_preds = pred_labels(messages=df_test.text)

In [ ]:
test_true_labels = df_test[class_names].values

In [ ]:
get_f1_score(test_true_labels, test_preds)

FinAL f1 =  0.93756490516823


## Результаты

*   FinAL f1 =  0.9371160296120874 (база)
*   FinAL f1 =  0.9374004523364328 (param_grid = {'C': [100., 10000.]})
*   FinAL f1 =  0.93756490516823 (param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
